In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import MeanAbsolutePercentageError 


csv_folder_path = '/Users/shubhibhadauriya/Desktop/Diss Project/osfstorage-archive (1)'  
image_base_path = '/Users/shubhibhadauriya/Desktop/osfstorage-archive'


csv_files = [f for f in os.listdir(csv_folder_path) if f.endswith('.csv')]

all_filenames = []
all_labels = []


def find_image_file(csv_filename, image_dir):
    parts = csv_filename.split('/')
    timestamp_id = parts[-1].split('-')[0]
    corrected_timestamp_id = timestamp_id.replace(':', '/')
    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if corrected_timestamp_id in file:
                return os.path.join(root, file)
    return None


def load_images(filenames, base_path):
    images = []
    for filename in filenames:
        img_path = find_image_file(filename, base_path)
        if img_path and os.path.exists(img_path):
            images.append(img_path)
        else:
            print(f"File not found for CSV entry: {filename}")
            images.append(None)  
    return np.array(images)


for csv_file in csv_files:
    csv_file_path = os.path.join(csv_folder_path, csv_file)
    data = pd.read_csv(csv_file_path)
    data = data.dropna()

    
    image_width, image_height = 960, 960  
    data['lx'] = data['lx'] / image_width
    data['ly'] = data['ly'] / image_height
    data['rx'] = data['rx'] / image_width
    data['ry'] = data['ry'] / image_height

    features = ['lx', 'ly', 'rx', 'ry']

    
    filenames = data['filename'].values
    images = load_images(filenames, image_base_path)
    labels = data[features].values

    
    valid_indices = [i for i, img in enumerate(images) if img is not None]
    images = images[valid_indices]
    labels = labels[valid_indices]

    
    all_filenames.extend(images)
    all_labels.extend(labels)


all_filenames = tf.convert_to_tensor(all_filenames, dtype=tf.string)
all_labels = tf.convert_to_tensor(all_labels, dtype=tf.float32)


dataset = tf.data.Dataset.from_tensor_slices((all_filenames, all_labels))


dataset = dataset.shuffle(buffer_size=len(all_filenames), seed=38)


sampled_dataset = dataset.take(1000)


train_size = int(0.6 * 1000)
val_size = int(0.2 * 1000)
test_size = 1000 - train_size - val_size

train_dataset = sampled_dataset.take(train_size)
val_test_dataset = sampled_dataset.skip(train_size)
val_dataset = val_test_dataset.take(val_size)
test_dataset = val_test_dataset.skip(val_size)


def load_and_preprocess_image(filename, label):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [299, 299])  
    image = image / 255.0  
    return image, label


train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)


augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(x):
    for layer in augmentation_layers:
        x = layer(x)
    return x


train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x), y))


train_dataset = train_dataset.cache().batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)


base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(299, 299, 3))  

for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.6)(x)  
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)  
x = Dropout(0.6)(x) 
x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)
predictions = Dense(4, activation='linear')(x)


model = Model(inputs=base_model.input, outputs=predictions)


def euclidean_distance(true_coords, pred_coords):
    return tf.sqrt(tf.reduce_sum(tf.square(true_coords - pred_coords), axis=-1))

def calculate_relative_error(y_true, y_pred):
    true_lx, true_ly, true_rx, true_ry = tf.split(y_true, 4, axis=-1)
    pred_lx, pred_ly, pred_rx, pred_ry = tf.split(y_pred, 4, axis=-1)

    d_left_eye = euclidean_distance(tf.concat([true_lx, true_ly], axis=-1), 
                                    tf.concat([pred_lx, pred_ly], axis=-1))
    d_right_eye = euclidean_distance(tf.concat([true_rx, true_ry], axis=-1), 
                                     tf.concat([pred_rx, pred_ry], axis=-1))

    w = euclidean_distance(tf.concat([true_lx, true_ly], axis=-1), 
                           tf.concat([true_rx, true_ry], axis=-1))

    relative_error_left = d_left_eye / w
    relative_error_right = d_right_eye / w
    relative_error = tf.maximum(relative_error_left, relative_error_right)
    
    return relative_error


optimizer = Adam(learning_rate=1e-3)  
model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['mae', MeanAbsolutePercentageError(name="mean_absolute_percentage_error"), calculate_relative_error])


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)  
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')


history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=100,  
                    callbacks=[reduce_lr, checkpoint])


model.load_weights('best_model.keras')




def plot_history(history, model_name):
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(history.history['mae'], label='train_mae')
    plt.plot(history.history['val_mae'], label='val_mae')
    plt.title(f'{model_name} MAE')
    plt.xlabel('Epochs')
    plt.ylabel('MAE')
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['mean_absolute_percentage_error'], label='train_mape')
    plt.plot(history.history['val_mean_absolute_percentage_error'], label='val_mape')
    plt.title(f'{model_name} MAPE')
    plt.xlabel('Epochs')
    plt.ylabel('MAPE')
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(history.history['calculate_relative_error'], label='train_relative_error')
    plt.plot(history.history['val_calculate_relative_error'], label='val_relative_error')
    plt.title(f'{model_name} Relative Error')


ImportError: cannot import name 'layers' from 'tensorflow.keras.layers' (/Users/shubhibhadauriya/anaconda3/lib/python3.10/site-packages/keras/api/_v2/keras/layers/__init__.py)

In [5]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, RandomFlip, RandomRotation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import MeanAbsolutePercentageError  # Import the MAPE metric


csv_folder_path = '/Users/shubhibhadauriya/Desktop/Diss Project/osfstorage-archive (1)'  
image_base_path = '/Users/shubhibhadauriya/Desktop/osfstorage-archive'


csv_files = [f for f in os.listdir(csv_folder_path) if f.endswith('.csv')]


all_filenames = []
all_labels = []


def find_image_file(csv_filename, image_dir):
    parts = csv_filename.split('/')
    timestamp_id = parts[-1].split('-')[0]
    corrected_timestamp_id = timestamp_id.replace(':', '/')
    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if corrected_timestamp_id in file:
                return os.path.join(root, file)
    return None


def load_images(filenames, base_path):
    images = []
    for filename in filenames:
        img_path = find_image_file(filename, base_path)
        if img_path and os.path.exists(img_path):
            images.append(img_path)
        else:
            print(f"File not found for CSV entry: {filename}")
            images.append(None)  
    return np.array(images)


for csv_file in csv_files:
    csv_file_path = os.path.join(csv_folder_path, csv_file)
    data = pd.read_csv(csv_file_path)
    data = data.dropna()

    
    image_width, image_height = 960, 960  
    data['lx'] = data['lx'] / image_width
    data['ly'] = data['ly'] / image_height
    data['rx'] = data['rx'] / image_width
    data['ry'] = data['ry'] / image_height

    features = ['lx', 'ly', 'rx', 'ry']

    
    filenames = data['filename'].values
    images = load_images(filenames, image_base_path)
    labels = data[features].values

    
    valid_indices = [i for i, img in enumerate(images) if img is not None]
    images = images[valid_indices]
    labels = labels[valid_indices]

    
    all_filenames.extend(images)
    all_labels.extend(labels)


all_filenames = tf.convert_to_tensor(all_filenames, dtype=tf.string)
all_labels = tf.convert_to_tensor(all_labels, dtype=tf.float32)


dataset = tf.data.Dataset.from_tensor_slices((all_filenames, all_labels))


dataset = dataset.shuffle(buffer_size=len(all_filenames), seed=38)


sampled_dataset = dataset.take(1000)


train_size = int(0.6 * 1000)
val_size = int(0.2 * 1000)
test_size = 1000 - train_size - val_size

train_dataset = sampled_dataset.take(train_size)
val_test_dataset = sampled_dataset.skip(train_size)
val_dataset = val_test_dataset.take(val_size)
test_dataset = val_test_dataset.skip(val_size)


def load_and_preprocess_image(filename, label):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [299, 299])  
    image = image / 255.0  
    return image, label


train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)


augmentation_layers = [
    RandomFlip("horizontal"),
    RandomRotation(0.1),
]


def data_augmentation(x):
    for layer in augmentation_layers:
        x = layer(x)
    return x


train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x), y))


train_dataset = train_dataset.cache().batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)


base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(299, 299, 3))  

for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.6)(x)  
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)  
x = Dropout(0.6)(x) 
x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)
predictions = Dense(4, activation='linear')(x)


model = Model(inputs=base_model.input, outputs=predictions)


def euclidean_distance(true_coords, pred_coords):
    return tf.sqrt(tf.reduce_sum(tf.square(true_coords - pred_coords), axis=-1))


epsilon = 1e-7  

def calculate_relative_error(y_true, y_pred):
    true_lx, true_ly, true_rx, true_ry = tf.split(y_true, 4, axis=-1)
    pred_lx, pred_ly, pred_rx, pred_ry = tf.split(y_pred, 4, axis=-1)

    d_left_eye = euclidean_distance(tf.concat([true_lx, true_ly], axis=-1), 
                                    tf.concat([pred_lx, pred_ly], axis=-1))
    d_right_eye = euclidean_distance(tf.concat([true_rx, true_ry], axis=-1), 
                                     tf.concat([pred_rx, pred_ry], axis=-1))

    
    w = euclidean_distance(tf.concat([true_lx, true_ly], axis=-1), 
                           tf.concat([true_rx, true_ry], axis=-1)) + epsilon

    relative_error_left = d_left_eye / w
    relative_error_right = d_right_eye / w
    relative_error = tf.maximum(relative_error_left, relative_error_right)
    
    return relative_error



optimizer = Adam(learning_rate=1e-3)  
model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['mae', MeanAbsolutePercentageError(name="mean_absolute_percentage_error"), calculate_relative_error])


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)  
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')


history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=100,  
                    callbacks=[reduce_lr, checkpoint])


model.load_weights('best_model.keras')




def plot_history(history, model_name):
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(history.history['mae'], label='train_mae')
    plt.plot(history.history['val_mae'], label='val_mae')
    plt.title(f'{model_name} MAE')
    plt.xlabel('Epochs')
    plt.ylabel('MAE')
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['mean_absolute_percentage_error'], label='train_mape')
    plt.plot(history.history['val_mean_absolute_percentage_error'], label='val_mape')
    plt.title(f'{model_name} MAPE')
    plt.xlabel('Epochs')
    plt.ylabel('MAPE')
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(history.history['calculate_relative_error'], label='train_relative_error')
    plt.plot(history.history['val_calculate_relative_error'], label='val_relative_error')
    plt.title(f'{model_name} Relative Error')



Epoch 1/100
19/19 [==============================] - 540s 26s/step - loss: 3.3264 - mae: 0.4809 - mean_absolute_percentage_error: 3363.8630 - calculate_relative_error: 934861.0000 - val_loss: 2.7028 - val_mae: 0.3458 - val_mean_absolute_percentage_error: 857.8730 - val_calculate_relative_error: 166310.1406 - lr: 0.0010
Epoch 2/100
19/19 [==============================] - 306s 16s/step - loss: 2.3714 - mae: 0.2545 - mean_absolute_percentage_error: 2605.7131 - calculate_relative_error: 687967.5625 - val_loss: 2.1126 - val_mae: 0.3375 - val_mean_absolute_percentage_error: 909.2706 - val_calculate_relative_error: 232243.3125 - lr: 0.0010
Epoch 3/100
19/19 [==============================] - 290s 16s/step - loss: 1.8455 - mae: 0.2228 - mean_absolute_percentage_error: 2531.0789 - calculate_relative_error: 624178.1875 - val_loss: 1.6763 - val_mae: 0.3169 - val_mean_absolute_percentage_error: 1018.7793 - val_calculate_relative_error: 237244.8750 - lr: 0.0010
Epoch 4/100
19/19 [=================

19/19 [==============================] - 231s 12s/step - loss: 0.0332 - mae: 0.1042 - mean_absolute_percentage_error: 3015.3115 - calculate_relative_error: 734416.0625 - val_loss: 0.0277 - val_mae: 0.0973 - val_mean_absolute_percentage_error: 2319.8054 - val_calculate_relative_error: 538418.3750 - lr: 0.0010
Epoch 52/100
19/19 [==============================] - 29032s 1612s/step - loss: 0.0329 - mae: 0.1034 - mean_absolute_percentage_error: 3031.8528 - calculate_relative_error: 737536.7500 - val_loss: 0.0272 - val_mae: 0.0943 - val_mean_absolute_percentage_error: 2343.3296 - val_calculate_relative_error: 544866.1250 - lr: 0.0010
Epoch 53/100
19/19 [==============================] - 33710s 17s/step - loss: 0.0326 - mae: 0.1032 - mean_absolute_percentage_error: 3029.0886 - calculate_relative_error: 737102.0000 - val_loss: 0.0271 - val_mae: 0.0976 - val_mean_absolute_percentage_error: 2319.7280 - val_calculate_relative_error: 537234.9375 - lr: 0.0010
Epoch 54/100
19/19 [==================

KeyboardInterrupt: 

In [ ]:
# Evaluate on the test data
#test_loss, test_mae, test_mape, test_relative_error = model.evaluate(test_dataset)
#print(f"Test Loss: {test_loss}")
#print(f"Test MAE: {test_mae}")
#print(f"Test MAPE: {test_mape}")
#print(f"Test Relative Error: {test_relative_error}")

In [ ]:

base_model.trainable = True


model.summary()


optimizer = Adam(learning_rate=1e-5)  
model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['mae', MeanAbsolutePercentageError(name="mean_absolute_percentage_error"), calculate_relative_error])


checkpoint_finetune = ModelCheckpoint('best_finetuned_model.keras', 
                                      monitor='val_loss', 
                                      save_best_only=True, 
                                      mode='min',
                                      verbose=1)


epochs = 50  
print("Fine-tuning the entire model")
history_finetune = model.fit(train_dataset,
                             validation_data=val_dataset,
                             epochs=epochs,
                             callbacks=[reduce_lr, checkpoint_finetune])


plot_history(history_finetune, 'InceptionResNetV2 Fine-Tuned')


In [ ]:
# Evaluate on the test data
#test_loss, test_mae, test_mape, test_relative_error = model.evaluate(test_dataset)
#print(f"Test Loss: {test_loss}")
#print(f"Test MAE: {test_mae}")
#print(f"Test MAPE: {test_mape}")
#print(f"Test Relative Error: {test_relative_error}")